**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sat Sep  5 21:40:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
 1573105787620BLmg3lVdjLnU9Ax9.pdf
 5.pdf
 6.pdf
 ams
 ams2
 ans.gdoc
 books
 Brochure.gdoc
 Cairo2015.pdf
'Colab Notebooks'
 course3_downloads
'Coursera 32J8SD7P5JYP (1).pdf'
'Coursera 32J8SD7P5JYP.pdf'
'Coursera 6DVX7ZQJH3A4 (1).pdf'
'Coursera 6DVX7ZQJH3A4 (2).pdf'
'Coursera 6DVX7ZQJH3A4.pdf'
'Coursera 95AG4WGT4XXA.pdf'
'Coursera TBP25FACMFMJ (1).pdf'
'Coursera TBP25FACMFMJ (2).pdf'
'Coursera TBP25FACMFMJ.pdf'
'Coursera UUU2HZZJS5H6 (1).pdf'
'Coursera UUU2H

**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14285, done.
remote: Total 14285 (delta 0), reused 0 (delta 0), pack-reused 14285
Receiving objects: 100% (14285/14285), 12.85 MiB | 18.85 MiB/s, done.
Resolving deltas: 100% (9748/9748), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [5]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [6]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [7]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [8]:
!echo "bike" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [9]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-09-05 21:43:40--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  9.66MB/s    in 17s     

2020-09-05 21:43:57 (9.31 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [10]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/221.jpg        
  inflating: data/obj/221.txt        
  inflating: data/obj/222.jpg        
  inflating: data/obj/222.txt        
  inflating: data/obj/223.jpg        
  inflating: data/obj/223.txt        
  inflating: data/obj/224.jpg        
  inflating: data/obj/224.txt        
  inflating: data/obj/225.jpg        
  inflating: data/obj/225.txt        
  inflating: data/obj/226.jpg        
  inflating: data/obj/226.txt        
  inflating: data/obj/227.jpg        
  inflating: data/obj/227.txt        
  inflating: data/obj/228.jpg        
  inflating: data/obj/228.txt        
 extracting: data/obj/classes.txt    
  inflating: data/obj/0.jpg          
  inflating: data/obj/0.txt          
  inflating: data/obj/1.jpg          
  inflating: data/obj/1.txt          
  inflating: data/obj/2.jpg          
  inflating: data/obj/2.txt          
  inflating: data/obj/3.jpg          
  inflating: data/obj/3.txt          
  inflating: 

In [11]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.490228 0.518293 0.700326 0.853659

['15', '0.490228', '0.518293', '0.700326', '0.853659']
0 data/obj/55.txt
0 0.490228 0.518293 0.700326 0.853659
15 0.510830 0.500000 0.732852 0.824176

['15', '0.510830', '0.500000', '0.732852', '0.824176']
1 data/obj/46.txt
0 0.510830 0.500000 0.732852 0.824176
15 0.777372 0.546196 0.437956 0.907609

['15', '0.777372', '0.546196', '0.437956', '0.907609']
3 data/obj/140.txt
0 0.777372 0.546196 0.437956 0.907609
15 0.518333 0.562500 0.776667 0.875000

['15', '0.518333', '0.562500', '0.776667', '0.875000']
4 data/obj/154.txt
0 0.518333 0.562500 0.776667 0.875000
15 0.510204 0.509756 0.922449 0.746341

['15', '0.510204', '0.509756', '0.922449', '0.746341']
5 data/obj/7.txt
0 0.510204 0.509756 0.922449 0.746341
15 0.485455 0.445355 0.905455 0.803279

['15', '0.485455', '0.445355', '0.905455', '0.803279']
6 data/obj/22.txt
0 0.485455 0.445355 0.905455 0.803279
15 0.483051 0.549708 0.545763 0.514620

['15', '0.483051', '0.549708', '0.545763', '0.514620'

In [12]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/123.jpg', 'data/obj/3.jpg', 'data/obj/226.jpg', 'data/obj/57.jpg', 'data/obj/193.jpg', 'data/obj/65.jpg', 'data/obj/102.jpg', 'data/obj/23.jpg', 'data/obj/44.jpg', 'data/obj/26.jpg', 'data/obj/51.jpg', 'data/obj/31.jpg', 'data/obj/164.jpg', 'data/obj/107.jpg', 'data/obj/103.jpg', 'data/obj/21.jpg', 'data/obj/130.jpg', 'data/obj/37.jpg', 'data/obj/148.jpg', 'data/obj/188.jpg', 'data/obj/195.jpg', 'data/obj/203.jpg', 'data/obj/115.jpg', 'data/obj/165.jpg', 'data/obj/6.jpg', 'data/obj/121.jpg', 'data/obj/35.jpg', 'data/obj/174.jpg', 'data/obj/161.jpg', 'data/obj/173.jpg', 'data/obj/106.jpg', 'data/obj/24.jpg', 'data/obj/218.jpg', 'data/obj/38.jpg', 'data/obj/217.jpg', 'data/obj/169.jpg', 'data/obj/42.jpg', 'data/obj/163.jpg', 'data/obj/14.jpg', 'data/obj/124.jpg', 'data/obj/43.jpg', 'data/obj/66.jpg', 'data/obj/216.jpg', 'data/obj/29.jpg', 'data/obj/63.jpg', 'data/obj/219.jpg', 'data/obj/113.jpg', 'data/obj/41.jpg', 'data/obj/101.jpg', 'data/obj/45.jpg', 'data/obj/5.jpg', 'data

In [13]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 